In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
class PairsTradingStrategy:
    def __init__(self, stock1_data, stock2_data, signals, initial_budget=100000):
        self.stock1_data = stock1_data
        self.stock2_data = stock2_data
        self.signals = signals
        self.initial_budget = initial_budget
        self.current_signal_index = 0
        self.budget = initial_budget
        self.inPosition = 0
        self.current_date = stock1_data.index[0]

    def next(self):
        signal = self.signals.iloc[self.current_signal_index]

        print("Date:", self.current_date, "Budget:", self.budget)
        #print(self.budget)

        
        if self.inPosition == 1:
            if signal.iloc[0] == -1:
                # Liquidate positions
                self.budget += self.stock1_data.iloc[self.current_signal_index, 0]
                self.budget += self.stock2_data.iloc[self.current_signal_index, 0]
                self.inPosition = 0
                
        elif self.inPosition == 0:
            if signal.iloc[0] == -1:
                # Enter new positions
                x = min(self.budget // 2, int(self.budget // (2 * self.stock1_data.iloc[self.current_signal_index, 0])))
                self.budget -= x * self.stock1_data.iloc[self.current_signal_index, 0]
                self.budget += x * self.stock2_data.iloc[self.current_signal_index, 0]
                self.inPosition = -1
                
            elif signal.iloc[0] == 1:
                # Enter new positions
                x = min(self.budget // 2, int(self.budget // (2 * self.stock1_data.iloc[self.current_signal_index, 0])))
                self.budget += x * self.stock1_data.iloc[self.current_signal_index, 0]
                self.budget -= x * self.stock2_data.iloc[self.current_signal_index, 0]
                self.inPosition = 1
                

        elif self.inPosition == -1:
            if signal.iloc[0] == 1:
                # Liquidate positions
                self.budget += self.stock1_data.iloc[self.current_signal_index, 0]
                self.budget += self.stock2_data.iloc[self.current_signal_index, 0]
                self.inPosition = 0
                

        self.current_signal_index += 1
        
        self.current_date = self.stock1_data.index[self.current_signal_index]

        if self.current_signal_index == len(self.signals) - 1:
            # Close all positions when all signals have been processed
            if self.inPosition == 1:
                self.budget += self.stock1_data.iloc[self.current_signal_index, 0]
                self.budget += self.stock2_data.iloc[self.current_signal_index, 0]
            elif self.inPosition == -1:
                self.budget += self.stock1_data.iloc[self.current_signal_index, 0]
                self.budget += self.stock2_data.iloc[self.current_signal_index, 0]

            self.inPosition = 0

        return self.budget

def run_backtest(stock1_data, stock2_data, signals, initial_budget):
    strategy = PairsTradingStrategy(stock1_data, stock2_data, signals, initial_budget)
    
    # Iterate over each signal to simulate the backtest
    for _ in range(len(signals)):
        final_budget = strategy.next()

    return final_budget

if __name__ == "__main__":
    # Assuming your data variable is available
    stock1_data = data[['Date', 'SWKS']].set_index("Date")
    stock2_data = data[['Date', 'QRVO']].set_index("Date")
    signals = data[['Date', 'Signal']].set_index("Date")

    initial_budget = 100

    final_budget = run_backtest(stock1_data, stock2_data, signals, initial_budget)

    print("Final Budget:", final_budget)
